In [1]:
import os

In [2]:
%pwd

'e:\\Retail-Analysis-with-Walmart\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Retail-Analysis-with-Walmart'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    train_data_path: Path
    test_data_path: Path


In [7]:
from real_walmart.constants import *
from real_walmart.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path
        )

        return data_transformation_config


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from real_walmart.logging import logger


In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def convert(self):
        # Load dataset
        df = pd.read_csv(self.config.data_path)

        # Separate features and target
        X = df.drop(columns=["Weekly_Sales", "Date"], errors="ignore")
        y = df["Weekly_Sales"]

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X,
            y,
            test_size=0.2,
            random_state=42
        )

        # Save transformed datasets
        train_df = pd.concat([X_train, y_train], axis=1)
        test_df = pd.concat([X_test, y_test], axis=1)

        train_df.to_csv(self.config.train_data_path, index=False)
        test_df.to_csv(self.config.test_data_path, index=False)

        logger.info("Data transformation completed successfully")


In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()

    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e


[2025-12-20 08:45:50,522: INFO: common: YAML file loaded successfully: config\config.yaml]
[2025-12-20 08:45:50,530: INFO: common: YAML file loaded successfully: params.yaml]
[2025-12-20 08:45:50,530: INFO: common: Created directory at: artifacts]
[2025-12-20 08:45:50,538: INFO: common: Created directory at: artifacts/data_transformation]
[2025-12-20 08:45:50,691: INFO: 2045458923: Data transformation completed successfully]
